# Nonogram Picture to Standard Form Output

In [1]:
import cv2
import numpy as np
import pytesseract

## Test Variables

In [2]:
img_for_box_extraction_path = "TestNonograms/Example4.png"
pytesseract.pytesseract.tesseract_cmd = r'#Insert your directory of Pytesseract here. To find look in the readme.md'

In [3]:
def createVerticalKernel(nonogram_image, ratio):
    return cv2.getStructuringElement(cv2.MORPH_RECT, (1, nonogram_image.shape[1]//ratio))
def createHorizontalKernel(nonogram_image, ratio):
    return cv2.getStructuringElement(cv2.MORPH_RECT, (nonogram_image.shape[0]//ratio, 1))

def isolateNonogram(nonogram_image):
    contours, hierarchy = cv2.findContours(nonogram_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    max_area = 0
    for contour in contours:
        
        area = cv2.contourArea(contour)
        peri = cv2.arcLength(contour, True)
        vertices = cv2.approxPolyDP(contour, 0.02 * peri, True)
        if area > max_area and len(vertices) == 4:
            biggest = vertices
            max_area = area
    return biggest, max_area

In [4]:
img = cv2.imread(img_for_box_extraction_path)
# Thresholding the image
imgGray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
img_bin = cv2.adaptiveThreshold(imgGray,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
img_bin = 255- img_bin 



# Create Kernels and apply them to isolate vertical and horizontal lines
kernel = createVerticalKernel(img_bin, 80)
vertical_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, kernel)

kernel = createHorizontalKernel(img_bin, 80)
horizontal_lines = cv2.morphologyEx(img_bin, cv2.MORPH_OPEN, kernel)

# Combine horizontal and vertical lines to find the nonogram area
# Weighting parameters, this will decide the quantity of an image to be added to make a new image.
alpha = 0.5
beta = 1.0 - alpha
# This function helps to add two image with specific weight parameter to get a third image as summation of two image.
img_final_bin = cv2.addWeighted(vertical_lines, alpha, horizontal_lines, beta, 0.0)
#img_final_bin = cv2.erode(~img_final_bin, kernel, iterations=2)
(thresh, img_final_bin) = cv2.threshold(img_final_bin, 128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

# Isolate the largest quadralaterel found, assumed to be the nonogram.
biggest, max_area = isolateNonogram(img_final_bin)
x,y,w,h = cv2.boundingRect(biggest)
isolated_bin_img = imgGray[y:y+h,x:x+w]
isolated_img = img[y:y+h,x:x+w]
isolated_gray_img = imgGray[y:y+h,x:x+w]


# Isolate cells with a certain gray that indicates digits.
numberCell = cv2.inRange(isolated_img, (208,208,208), (208,208,208))
# Find the square contour of the box
cnts, _ = cv2.findContours(numberCell, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Assuming they are in order right to left top to bottom
# MF it literally goes right to left bottom to top :DDDD imagine!
# We save the x coord of first image and 

# Since starts bottom right we know this is furtherst right a num can be without being top!
# Means if x goes higher than this becomes a top row item!
max_x_without_being_top,_,_,_ = cv2.boundingRect(cnts[0])
horizontal_dictionary = {}
vertical_dictionary   = {}
# We need to keep track of last x and last y 
last_y = -2
# We should keep a boolean for if we have switched from horizontal to vertical yet.
switch_to_vertical_strip_yet = False
# Binarise the text?
img_bin_bin = cv2.adaptiveThreshold(isolated_bin_img,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
for c in cnts:
    x,y,w,h = cv2.boundingRect(c)
    
    number = img_bin_bin[y:y+h,x:x+w]
    # Need to up the resolution of the number becasue pytesseract misclassifies at lower
    number = cv2.pyrUp(number)
    number = cv2.pyrUp(number)
    # Scan box for the number store as "data"
    data = pytesseract.image_to_string(number, lang="eng", config='--psm 13 --oem 3 -c tessedit_char_whitelist=0123456789')
    
    
    if switch_to_vertical_strip_yet or max_x_without_being_top < x:
        switch_to_vertical_strip_yet = True
        # We are a vertical gang so act like it.
        dictionary_storage = vertical_dictionary.get(x)
        if dictionary_storage is not None:
            vertical_dictionary[x].append(int(data))
        else:
            vertical_dictionary[x] = [int(data)]
    else:
        # Still on horizontal.
        # Check not a different row from last
        if last_y == y:
            # Not the first, add to the dictionary.
            horizontal_dictionary[y].append(int(data))
        else:
            horizontal_dictionary[y] = [int(data)]
            last_y = y
            
            
            
# Ok now we have to dictionaries filled with values
# We just need to reverse them! and plop them out!
# Horizontal clues first
horizontal_locations = list(horizontal_dictionary.keys())
horizontal_locations.sort()
horizontal_clues = []
for location in horizontal_locations:
    horizontal_dictionary[location].reverse()
    horizontal_clues.append(horizontal_dictionary[location])

# Same for vertical!
vertical_locations = list(vertical_dictionary.keys())
vertical_locations.sort()
vertical_clues = []
for location in vertical_locations:
    vertical_dictionary[location].reverse()
    vertical_clues.append(vertical_dictionary[location])


print(horizontal_clues)
print(vertical_clues)


cv2.drawContours(isolated_img, cnts, -1, (0,255,0))
cv2.imwrite("example_with_bounding_boxes.jpg", isolated_img)




cv2.imshow('Isolated', isolated_img)
cv2.waitKey()

[[1], [3], [1, 1, 1], [1], [4], [6, 3], [8, 2], [2, 8, 3], [13], [11], [13]]
[[1], [3, 1], [2, 3], [1, 7], [1, 7], [11], [1, 7], [1, 6], [5], [4], [4], [3], [4], [2, 1], [3], [2], [1]]


-1